# Naive Bayes

## Introducción

En Python existen múltiples formas de aplicar machine learning para el reconocimiento de patrones. Una de estas consiste en los clasificadores probabilísticos, entre los cuales encontramos el __teorema de Bayes.__

Bayes se caracteriza por asumir una gran independencia entre las características que se encuentran en los datos, lo cual ofrece una perspectiva distinta de el problema que se va a resolver: Obtener la categoría del crimen dada su localización en distrito, hora, día de la semana y mes.

## Librerías

Se utilizará pandas y sklearn para poder utilizar los dataframes y modelos necesarios. Pprint para visualización.

In [10]:
import pandas as pd
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB
import pprint

## Recuperación y formato de datos

En primer lugar obtenemos el CSV en partes. Se realizará un filtrado de fechas y tiempo (hora) para poder trabajar con los datos. Las dos partes se obtienen para separar un conjunto de entrenamiento, y otro de testeo, que hará de entrada simulada en este caso.

In [3]:
url = 'https://storage.googleapis.com/grupospark/incidents.all.ordered.date.csv'
test = pd.read_csv(url, parse_dates=['Date'], header=0, nrows=10000)
test['Time'] = pd.to_datetime(test['Time'],format='%H:%M')
train = pd.read_csv(url, parse_dates=['Date'],  header=0,skiprows=range(1,10000),nrows=90000)
train['Time'] = pd.to_datetime(train['Time'],format='%H:%M')

## Entrenamiento y tests

Bayes es una técnica cuya clasificación puede ser entrenada de forma muy eficiente siempre que sea de forma __supervisada__, es por ello que se normalizarán los datos para entrenarlos o procesarlos. Se realiza un pre-procesamiento con las siguientes funciones. 

In [5]:
def normalize(data): # Normalización de características.
    data = (data - data.mean()) / (data.max() - data.min())
    return data

# Preparación de datos
def prep_data(data, test):
    if (test == 0):
        # Se realiza una codificación de las etiquetas
        crimen_labels = preprocessing.LabelEncoder()
        crimen_encode = crimen_labels.fit_transform(data.Category)

    # Con get_dummies convertimos nuestros valores categóricos en valores númericos con los que poder trabajar
    days = pd.get_dummies(data.DayOfWeek)
    district = pd.get_dummies(data.PdDistrict)
    month = pd.get_dummies(data.Date.dt.month,prefix="m")
    hour = data.Time.dt.hour
    hour = pd.get_dummies(hour)

    # Construimos el array a partir de los datos obtenidos
    prepared_data = pd.concat([hour, month, days, district], axis=1)
    prepared_data['X'] = normalize(data.X)
    prepared_data['Y'] = normalize(data.Y)

    if (test == 0):
        prepared_data['crime'] = crimen_encode

    return prepared_data

# Preprocesamiento llevado a cabo
train_proc = prep_data(train, 0)
test_proc = prep_data(test, 1)

## Características a utilizar

Es esencial, en este algoritmo, especificar qué valores obtenidos desde la fuente de datos queremos utilizar como punto de partida para la clasificación. En la función anterior distinguimos los meses (con prefijo "m_") de las horas (el rango numérico de 0 a 23). 

In [7]:
features = [  'm_1', 'm_2', 'm_3', 'm_4', 'm_5', 'm_6', 'm_7', 'm_8', 'm_9', 'm_10', 'm_11', 'm_12',
               'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
               'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN',
               'TARAVAL', 'TENDERLOIN'
           ] + [x for x in range(0, 24)]

## Bayes y predicciones

Con el modelo de Bernoulli, que equivale al modelo binario de independencia, realizamos nuestro modelo a partir del conjunto de entrenamiento ya preparado. Tras esto, con el mismo modelo intentamos predecir los casos de test.

In [8]:
model = BernoulliNB()
model.fit(train_proc[features], train_proc['crime'])
predicted = model.predict_proba(test_proc[features])

## Resultados y análisis

Por último, exponemos el resultado de nuestro proceso. Dado que estamos en el repositorio, comentado el código para almacenar este en un CSV. En su lugar convertimos este a un listado de diccionarios sobre el que podemos iterar para obtener los tres primeros casos de test.

In [15]:
# Write results
crimen_labels = preprocessing.LabelEncoder()
crimen = crimen_labels.fit_transform(train.Category)
result = pd.DataFrame(predicted, columns=crimen_labels.classes_)
#result.to_csv('results.csv', index=True, index_label='Id'

result_dict = result.to_dict(orient='records')

# Primer caso de test, para visualizar la estructura
pprint.pprint(result_dict[0])
print('\n')
# Iteramos sobre los tres primeros tests, para visualizar cuál es el valor máximo de cada resultado
for i, row in enumerate(result_dict):
    maximo = max(row, key=lambda key: row[key])
    print('Caso',i,': Categoría-> ',maximo, '| Resultado-> ',row[maximo])
    if i == 2:
        break;





{'ARSON': 0.034332225834823565,
 'ASSAULT': 0.047551791152962736,
 'BAD CHECKS': 0.0028829952318701405,
 'BRIBERY': 0.013526986958289225,
 'BURGLARY': 0.02731243879623048,
 'DISORDERLY CONDUCT': 0.004351904024434736,
 'DRIVING UNDER THE INFLUENCE': 0.005169667320699265,
 'DRUG/NARCOTIC': 0.0993090885750867,
 'DRUNKENNESS': 0.0019527956268837066,
 'EMBEZZLEMENT': 0.012016693387333568,
 'EXTORTION': 0.0017078669536744534,
 'FAMILY OFFENSES': 0.0036255612759995286,
 'FORGERY/COUNTERFEITING': 0.026312768999884172,
 'FRAUD': 0.01038061964305988,
 'GAMBLING': 0.0070507821226408985,
 'KIDNAPPING': 0.012860688585998665,
 'LARCENY/THEFT': 0.022770216117140567,
 'LIQUOR LAWS': 0.005433796665775091,
 'LOITERING': 0.014457488665450862,
 'MISSING PERSON': 0.021583838781292233,
 'NON-CRIMINAL': 0.028694929727619296,
 'OTHER OFFENSES': 0.12787248134197488,
 'PORNOGRAPHY/OBSCENE MAT': 4.1608133663723245e-07,
 'PROSTITUTION': 5.20058065245541e-05,
 'ROBBERY': 0.012147180452482009,
 'RUNAWAY': 0.0045594

## Conclusión

Podemos observar que el objeto resultado es un diccinario de todas las posibles categorías de crímenes con sus respectivas probabilidades.

En los tres primeros casos de ejemplo, tenemos un __30%__ de probabilidad de que sea __WARRANTS__, en el segundo, un __17%__, y en el tercero, un __19%__ de que sea un robo. 

El procesamiento por Bayes no es del todo certero, pero sí veloz gracias a su forma de procesar las categorías, siendo un buen candidato para este tipo de predicciones.